In [1]:
%ls /var/data/paper_data/

data_docker.pkl  data_prod.pkl  data_prod_test.pkl


In [2]:
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score, precision_score, confusion_matrix


pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 500)
# import sys
# sys.path.append('michael/deeplearn.det/deeplearn')  
# from model import evaluation_print
# from michael//deeplearn.det//deeplearn//model import evaluation_print

In [3]:
import numpy as np
a, b, c=np.ones((2,8)), np.ones((2,8)), np.ones((2,8))
np.vstack([a,b,c]).shape

(6, 8)

In [4]:
def string_preproc(a):
    """
       input string
       output string without not relevant stuff
       example: input "mailservice_nimol_server_qmgr.nimol.hist"
                output "mailservice\_server\_qmgr"

    """

    spl = a.split('_')
    return '\_'.join([spl[0], spl[2], spl[3].split('.')[0]])

def top_n_predicted(probs, true_labels, top_n_value):
    """
    input: probabilites per 'point', and true_labels, and top_n value
    output: labels for predictions. If the true probabilit are within top_n values
    it returns the true label in its output, otherwise it returns the argmax.
    """
    predicted_labels_top = []

    for line, true_label in zip(probs, true_labels):
        sorted_probs = np.argsort(line)[::-1][:top_n_value]
        if true_label in sorted_probs:
            predicted_labels_top.append(true_label)
        else:
            predicted_labels_top.append(sorted_probs[0])


    return predicted_labels_top

def top_n(vals, names, top_n_val):
    top_sorted_indexes = np.argsort(vals)[::-1][:top_n_val]
    return [names[i] for i in top_sorted_indexes]

def evaluation_print(true_labels, prediction, process_names, top_n_value=1):

#     if top_n > 1:
#         # print prediction
#         predicted_labels = top_n_predicted(prediction, true_labels, top_n_value=top_n_value)
#         # predicted_labels
#     elif top_n_value == 1:
#         predicted_labels = prediction
    predicted_labels = prediction
    np.set_printoptions(precision=6, suppress=True)
    np.set_printoptions(threshold=10000, linewidth=1000)
    conf_matrix = confusion_matrix(true_labels, predicted_labels)

#     process_names = [string_preproc(nam) for nam in process_names]
    print process_names
    print 'top_n = ', top_n_value
    # print conf_matrix

    n = conf_matrix.shape[0]
    print pd.DataFrame(conf_matrix, index=range(n), columns=range(n))

    print 'macro average recall', '%.2f' % recall_score(true_labels, predicted_labels, average='macro')
    print 'macro average precision', '%.2f' % precision_score(true_labels, predicted_labels, average='macro')
    print '\\begin{tabular} {rrr}'
    print '\\hline'
    for i, (name, recall, precision) in enumerate(zip(
                            process_names,
                            recall_score(true_labels, predicted_labels, average=None),
                            precision_score(true_labels, predicted_labels, average=None)
                                      )):
        print i, name, '&', '%.4f' % recall, '&', '%.4f' % precision, '\\\\'
        # print name, '%.4f' % recall, '%.4f' % precision

    print '\\hline'
    print '\\end{tabular}'

    print '\n' * 2
#     conf_matrix = np.hstack([np.array(range(n), ndmin=2).T, conf_matrix])
#     conf_matrix = np.vstack([[0] + range(n), conf_matrix])
#     print(tabulate(conf_matrix, tablefmt="latex"))


In [5]:
%time data_train, proc_names = joblib.load('/var/data/paper_data/data_prod.pkl') 

CPU times: user 35.8 s, sys: 2.24 s, total: 38 s
Wall time: 38 s


In [6]:
data_blocks_train = [d for _, d in data_train]
labels_train = [np.argmax(l) for l, _ in data_train]
labels_train = [[l] * 10 for l in labels_train]
labels_train = [item for sublist in labels_train for item in sublist]
data_sum_train = [d.sum(axis=0) for d in data_blocks_train]
data_concat_train = [d.ravel() for d in data_blocks_train]
data_lines_train = np.vstack(data_blocks_train)

In [7]:
%time data_test, proc_names = joblib.load('/var/data/paper_data/data_prod_test.pkl') 

CPU times: user 53.4 s, sys: 3.2 s, total: 56.6 s
Wall time: 56.6 s


In [8]:
data_blocks_test = [d for _, d in data_test]
labels_test = [np.argmax(l) for l, _ in data_test]
labels_test = [[l] * 10 for l in labels_test]
labels_test = [item for sublist in labels_test for item in sublist]

data_sum_test = [d.sum(axis=0) for d in data_blocks_test]
data_concat_test = [d.ravel() for d in data_blocks_test]
data_lines_test = np.vstack(data_blocks_test)

In [12]:
clf = LogisticRegression(n_jobs=12, verbose=5)
# clf = LinearSVC(C=1)
# clf = RandomForestClassifier(n_jobs=12)
# clf = DecisionTreeClassifier()

In [13]:
clf.fit(data_lines_train, labels_train)

[LibLinear]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=12,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=5, warm_start=False)

In [15]:
data_lines_test.shape

(5719700, 324)

In [16]:
prediction = clf.predict(data_lines_test)

In [ ]:
n = 2 * 10 **6

In [20]:
np.set_printoptions(precision=6, suppress=True)
np.set_printoptions(threshold=10000, linewidth=10000, edgeitems=10000)
# print confusion_matrix(prediction, labels_test)
print 'macro average recall', '%.2f' % recall_score(labels_test, prediction, average='macro')
print 'macro average precision', '%.2f' % precision_score(labels_test, prediction, average='macro')
for i, (name, recall, precision) in enumerate(zip(
                            proc_names,
                            recall_score(labels_test, prediction, average=None),
                            precision_score(labels_test, prediction, average=None)
                                      )):
        print i, name, '%.4f' % recall, '%.4f' % precision

macro average recall 0.74
macro average precision 0.79
0 iscsid 1.0000 0.6689
1 httpd 0.0000 0.0000
2 cyagateway 1.0000 0.9992
3 hsflowd 0.3476 0.9845
4 ntpd 0.9999 0.6042
5 bender 0.0075 0.9970
6 cyacollector 0.9908 0.9999
7 caldaemon 1.0000 1.0000
8 appdisp 0.0000 0.0000
9 syslog-ng 0.9994 0.9995
10 java 1.0000 0.9999
11 wrapper 1.0000 1.0000
12 cma 0.0000 0.0000
13 perfstat 0.9997 0.9998
14 add_time.pl 0.9954 1.0000
15 init 0.9998 1.0000
16 tail 0.9998 1.0000
17 svscan 0.9982 0.9925


In [19]:
evaluation_print(true_labels=labels_test, prediction=prediction, process_names=proc_names,top_n_value=1)

['iscsid', 'httpd', 'cyagateway', 'hsflowd', 'ntpd', 'bender', 'cyacollector', 'caldaemon', 'appdisp', 'syslog-ng', 'java', 'wrapper', 'cma', 'perfstat', 'add_time.pl', 'init', 'tail', 'svscan', 'yamproxy', 'nscd']
top_n =  1
        0   1       2       3       4     5       6       7   8       9       10      11  12      13      14     15     16     17
0   991335   0       0       0       0     7       6       0   0       0       0       0   2       0       0      0      0      0
1        0   0       0       0       0     0       0       0   0       0       0       0   0       0       0      0      0      0
2        0   0  496897       0       0     0       0       0   0       0       9       0   0       0       0      0      0      4
3        0   0       0  172740  324150     0       0       0   0       0       0       0   0       0       0      0      0      0
4        0   0       0      64  494796     0       0       0   0       0       0       0   0       0       0      0      0  

In [26]:
r, p = [], []
for i in range(10):
#     clf = LogisticRegression(n_jobs=20, verbose=2)
#     clf = LinearSVC(C=1, verbose=2)
    clf = RandomForestClassifier(n_jobs=20, verbose=2)

    clf.fit(data_lines_train, labels_train)
    prediction = clf.predict(data_lines_test)
    # print confusion_matrix(prediction, test_label)
    r.append(recall_score(labels_test, prediction, average='macro'))
    p.append(precision_score(labels_test, prediction, average='macro'))
    print i

building tree 1 of 10
 building tree 3 of 10
building tree 2 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
 building tree 8 of 10building tree 7 of 10
 building tree 10 of 10
building tree 9 of 10



[Parallel(n_jobs=20)]: Done   7 out of  10 | elapsed:   29.4s remaining:   12.6s
[Parallel(n_jobs=20)]: Done  10 out of  10 | elapsed:   34.2s finished
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:    2.2s remaining:    5.2s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    4.5s finished


0
building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=20)]: Done   7 out of  10 | elapsed:   28.8s remaining:   12.3s
[Parallel(n_jobs=20)]: Done  10 out of  10 | elapsed:   33.3s finished
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:    2.3s remaining:    5.3s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    4.7s finished


1
building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


...

7
building tree 1 of 10building tree 2 of 10

building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=20)]: Done   7 out of  10 | elapsed:   32.5s remaining:   13.9s
[Parallel(n_jobs=20)]: Done  10 out of  10 | elapsed:   37.3s finished
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:    2.3s remaining:    5.3s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    4.1s finished


8
building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=20)]: Done   7 out of  10 | elapsed:   31.8s remaining:   13.6s
[Parallel(n_jobs=20)]: Done  10 out of  10 | elapsed:   34.1s finished
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:    2.4s remaining:    5.6s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    3.7s finished


9


In [27]:
np.set_printoptions(suppress=True, precision=3)
print '%.3f' % np.mean(p), np.std(p), '%.3f' % np.mean(r), np.std(r)

0.850 0.0209846673912 0.795 0.0196401496198


### LR
0.791 0.0 0.741 1.11022302463e-16

### Lin SVM

0.792 1.11022302463e-16 0.741 0.0

### RF

0.850 0.0209846673912 0.795 0.0196401496198

In [ ]:
##